In [6]:
import nibabel as nib
import numpy as np 
import matplotlib.pyplot as plt
import math
import cv2
import os,glob,re,time
import pandas as pd
from PIL import Image
import matplotlib as mpl
import tkinter as tk
from tkinter import ttk
import pathlib
from tkinter import messagebox

In [ ]:
def load_data():
    RdWh = mpl.colors.ListedColormap([(0,0,0,0),(1,0,0,1)])
    in_path = in_path_box.get()
    out_path = out_path_box.get()
    brain1 = []
    brain2 = []
    path_nii = in_path 
    fig = plt.figure()
    file_kazu=len(os.listdir(path_nii))
    cou=0
    try:
        for file_number in range(file_kazu):
            img_file = glob.glob(path_nii+"/*.nii")
            path=os.path.join(img_file[int(cou)])
            nib_path = nib.load(path)
            img_data = nib_path.get_fdata().T[::-1]
            img2 =img_data[20]
            ret,thresh1 = cv2.threshold(img2,3,255,cv2.THRESH_BINARY)
            kara=np.zeros_like(thresh1)

            
            brain1_lis = []
            brain2_lis = []
            
            switch1=0
            switch2=1
            for i, img in enumerate(img_data):
                brain1_lis.append(img)
                ret,thresh1=cv2.threshold(img,3,255,cv2.THRESH_BINARY)
                if 5<img.max():
                    switch1=1
                if 5<img.max() and switch1==1 and switch2==1:
                    brain2_lis.append(thresh1)
                else:
                    brain2_lis.append(kara)
                    if switch1==1 and i>10:
                        switch2=0
            arr_brain1 = np.array(brain1_lis)
            arr_brain2 = np.array(brain2_lis)
            brain1.append(arr_brain1)
            brain2.append(arr_brain2)
            
            fig = plt.figure()            
            arr1 = np.max(brain1[file_number],axis=1)
            arr2 = np.max(brain2[file_number],axis=1)
            plt.axis("off")
            plt.imshow(arr1,cmap="gray_r")
            plt.imshow(arr2,cmap=RdWh,alpha=0.5)
            fig.savefig(out_path+"\\"+str(cou),bbox_inches='tight', pad_inches=0)
            cou +=1
            

        time.sleep(2)
        
        messagebox.showinfo("完了", "セグメンテーションが完了しました。")
        in_path_box.delete(0, tk.END)
        out_path_box.delete(0, tk.END)

    except:
        messagebox.showerror('エラー', 'エラー発生が発生しました。処理を終了します。\r\n入力した値を確認してください。')
        in_path_box.delete(0, tk.END)
        out_path_box.delete(0, tk.END)


main_win = tk.Tk()
main_win.title("脳のセグメンテーションツール")
main_win.geometry("360x160")

frame1 = ttk.Frame(main_win)
frame1.grid(column=0, row=0, sticky=tk.NSEW, padx=5, pady=10)

in_path_label = ttk.Label(frame1, text="NIfTIファイル")
in_path_box = ttk.Entry(frame1)
in_path_box.insert(tk.END, "")

out_path_label = ttk.Label(frame1, text="出力先")
out_path_box = ttk.Entry(frame1)
out_path_box.insert(tk.END, "")

app_btn = ttk.Button(frame1, text="実行", width=12, command=load_data)

in_path_label.grid(column=0, row=0, pady=10)
in_path_box.grid(column=1, row=0, sticky=tk.EW, padx=5)
out_path_label.grid(column=0, row=1, pady=10)
out_path_box.grid(column=1, row=1, sticky=tk.EW, padx=5)
app_btn.grid(column=1, row=3, padx=10)

main_win.columnconfigure(0, weight=1)
main_win.rowconfigure(0, weight=1)
frame1.columnconfigure(1, weight=1)

main_win.mainloop() 